In [2]:
import sys
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\change_of_variable')
sys.path.append('F:\Cambridge\Project\MHMC-for-VAE\hmc_pytorch')
from change_of_variable_pytorch import * 
from hmc_base_pytorch import *
from hmc_unconstrained_pytorch import *
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal
import matplotlib.pyplot as plt

import torch.utils.data
from torch import optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
import time

In [3]:
def multivariate_normal_diagonal_logpdf(mean, cov_diag, x):
    mean = mean.cuda()
    cov_diag = cov_diag.cuda()
    n = x.shape[0] # number of samples
    k = x.shape[1] # dimension
    t1 = -0.5*(x - mean)*(1/cov_diag)*(x-mean)
    t1 = torch.sum(t1, dim=1)
    #t2 = 0.5*k*torch.log(2*torch.tensor([math.pi]).cuda()) + 0.5*torch.log(torch.prod(cov_diag,1)).cuda()
    t2 = 0.5*k*torch.log(2*torch.tensor([math.pi]).cuda()) + 0.5*torch.sum(torch.log(cov_diag)).cuda()
    return t1 - t2
    #return t1 - t2

In [4]:
mean = torch.tensor([[0.,0.,0.],[0.,0.,0.]]).cuda()
cov_diag = torch.tensor([[1.,1.,1.],[1.,1.,1.]]).cuda()
x = torch.tensor([[5.,0.5,0.],[1.,1.,0.]]).cuda()
multivariate_normal_diagonal_logpdf(mean, cov_diag, x)

tensor([-15.3818,  -3.7568], device='cuda:0')

In [20]:
mean.shape
torch.diag(cov_diag[0])

tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.]], device='cuda:0')

In [ ]:
def energy(y,cache):
    mean = 
    multivariate_normal_diagonal_logpdf(mean, cov_diag, x)

In [15]:
mean = torch.tensor([[1.,0.,0.],[0.,0.,0.]], requires_grad=True)
mean.shape

torch.Size([2, 3])

In [18]:
res = torch.sum(mean, dim=1)
res.shape

torch.Size([2])

In [17]:
res.backward()

RuntimeError: grad can be implicitly created only for scalar outputs